In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-965a134e-30db-fabe-9219-5aad00dc47cd)


In [ ]:
!pip install transformers Pillow torch requests

In [ ]:
from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration
import torch
from PIL import Image
import requests
import json
import random
import time
import os

In [ ]:
model_path = "/content/drive/MyDrive/ML Final Project/Models/instructblip-vicuna-7b"
processor_path = "/content/drive/MyDrive/ML Final Project/Processor/instructblip-vicuna-7b"

if not os.path.exists(model_path):
    model = InstructBlipForConditionalGeneration.from_pretrained("Salesforce/instructblip-vicuna-7b")
    model.save_pretrained("/content/drive/MyDrive/ML Final Project/Models/instructblip-vicuna-7b")

else:
    model = InstructBlipForConditionalGeneration.from_pretrained(model_path)

if not os.path.exists(processor_path):
    processor = InstructBlipProcessor.from_pretrained("Salesforce/instructblip-vicuna-7b")
    processor.save_pretrained("/content/drive/MyDrive/ML Final Project/Processor/instructblip-vicuna-7b")
else:
    processor = InstructBlipProcessor.from_pretrained(processor_path)

config.json:   0%|          | 0.00/6.66k [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/107k [00:00<?, ?B/s]

pytorch_model-00001-of-00004.bin:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

pytorch_model-00002-of-00004.bin:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

pytorch_model-00003-of-00004.bin:   0%|          | 0.00/9.92G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/442 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/209 [00:00<?, ?B/s]

In [ ]:
model.save_pretrained("/content/drive/MyDrive/ML Final Project/Models/instructblip-vicuna-7b")
processor.save_pretrained("/content/drive/MyDrive/ML Final Project/Processor/instructblip-vicuna-7b")

In [ ]:
model = InstructBlipForConditionalGeneration.from_pretrained("/content/drive/MyDrive/ML Final Project/Models/instructblip-vicuna-7b")
processor = InstructBlipProcessor.from_pretrained("/content/drive/MyDrive/ML Final Project/Processor/instructblip-vicuna-7b")

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
device = "cpu" if torch.cuda.is_available() else "cpu"
#device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

InstructBlipForConditionalGeneration(
  (vision_model): InstructBlipVisionModel(
    (embeddings): InstructBlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (encoder): InstructBlipEncoder(
      (layers): ModuleList(
        (0-38): 39 x InstructBlipEncoderLayer(
          (self_attn): InstructBlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1408, out_features=4224, bias=True)
            (projection): Linear(in_features=1408, out_features=1408, bias=True)
          )
          (layer_norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
          (mlp): InstructBlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1408, out_features=6144, bias=True)
            (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          )
          (layer_norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        

In [ ]:
url = "https://raw.githubusercontent.com/salesforce/LAVIS/main/docs/_static/Confusing-Pictures.jpg"
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")
prompt = "Generate caption"
inputs = processor(images=image, text=prompt, return_tensors="pt").to(device)

outputs = model.generate(
        **inputs,
        do_sample=False,
        num_beams=5,
        max_length=128,
        min_length=1,
        top_p=0.9,
        repetition_penalty=1.5,
        length_penalty=1.0,
        temperature=0,
)
generated_text = processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()
print(generated_text)

a man in a yellow shirt is ironing clothes on the back of a car


In [53]:
import io
# random.seed(42)

with open('/content/drive/MyDrive/ML Final Project/captions_val2014.json', 'r') as file:
    data = json.load(file)

beg = 270
threshold = 288
images_data = data.get('images', [])

# random.shuffle(images_data)
subset_size = len(images_data) // 5
selected_images = images_data[beg:subset_size]
#print(f"subset size: {subset_size}")
generated_captions = []

count = beg
for image_info in selected_images:
    start = time.time()
    image_url = image_info.get('url')

    image_id = image_info.get('id')
    resp = requests.get(image_url, stream=True)
    #print(resp.raw)
    if not resp.ok:
        #print(f"corrupt url : {image_url}")
        continue
    else:
        #image = Image.open(requests.get(image_url, stream=True).raw).convert("RGB")
        image = Image.open(resp.raw).convert("RGB")
    #print(image_id)
    prompt = "Generate caption"
    inputs = processor(images=image, text=prompt, return_tensors="pt").to(device)

    outputs = model.generate(
        **inputs,
        do_sample=False,
        num_beams=5,
        max_length=128,
        min_length=1,
        top_p=0.9,
        repetition_penalty=1.5,
        length_penalty=1.0,
        temperature=0,
        )
    generated_text = processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()

    image_id = image_info.get('id')
    generated_captions.append({"image_id": image_id, "caption": generated_text})
    count = count+1
    end = time.time()
    print(f"Processed Image: {count}/{threshold+1} \t Time Taken:{end-start}s")
    with open('/content/drive/MyDrive/ML Final Project/captions_instructblip_vicuna_results_updated17.json', 'w') as result_file:
        json.dump(generated_captions, result_file)
    if count > threshold:
        break

Processed Image: 271/289 	 Time Taken:42.16971564292908s
Processed Image: 272/289 	 Time Taken:43.72572636604309s
Processed Image: 273/289 	 Time Taken:46.29782962799072s
Processed Image: 274/289 	 Time Taken:50.52048087120056s
Processed Image: 275/289 	 Time Taken:46.44968771934509s
Processed Image: 276/289 	 Time Taken:48.19603872299194s
Processed Image: 277/289 	 Time Taken:47.299875020980835s
Processed Image: 278/289 	 Time Taken:48.62023639678955s
Processed Image: 279/289 	 Time Taken:42.84902882575989s
Processed Image: 280/289 	 Time Taken:50.49744939804077s
Processed Image: 281/289 	 Time Taken:44.35987424850464s
Processed Image: 282/289 	 Time Taken:46.29393649101257s
Processed Image: 283/289 	 Time Taken:49.75093460083008s
Processed Image: 284/289 	 Time Taken:48.84226131439209s
Processed Image: 285/289 	 Time Taken:55.738383531570435s
Processed Image: 286/289 	 Time Taken:49.4538254737854s
Processed Image: 287/289 	 Time Taken:48.35456418991089s
Processed Image: 288/289 	 Tim